In [1]:
#Import Packages
import os
import pandas as pd
import numpy as np
import rasterio as rio

#Change Working Directory
os.chdir(r'C:\Users\malva\Thesis\GIS\Layers\Raster\Bergueda')

#Read Rasters
lat = rio.open("latitude.asc")
long = rio.open("longitude.asc")

dem = rio.open('dem.asc')
slope = rio.open("slope.asc")
aspect = rio.open("aspect.asc")
curv_plan = rio.open("curvature_plan.asc")
curv_prof = rio.open("curvature_profile.asc")
facc = rio.open("facc.asc")
twi = rio.open("twi.asc")

soil = rio.open("soil.asc")
lulc = rio.open("lulc.asc")

#rain_ant = rio.open("rain_ant.asc")
#rain_event = rio.open("rain_event.asc")

#pof_event = rio.open("PROB_failure_final_cond.asc")
#pof_dry = rio.open("PROB_uncond_unst.asc")
#pof_sat = rio.open("PROB_uncond_stable.asc")

inventory = rio.open("inventory.asc")

In [2]:
%%time
#Convert Rasters to 2D Arrays
lat_arr = lat.read(1)
long_arr = long.read(1)

dem_arr = dem.read(1)
slp_arr = slope.read(1)
asp_arr = aspect.read(1)
curv_plan_arr = curv_plan.read(1)
curv_prof_arr = curv_prof.read(1)
facc_arr = facc.read(1)
twi_arr = twi.read(1)

soil_arr = soil.read(1)
lulc_arr = lulc.read(1)

#rain_ant_arr = rain_ant.read(1)
#rain_event_arr = rain_event.read(1)

#pof_event_arr = pof_event.read(1)
#pof_dry_arr = pof_dry.read(1)
#pof_sat_arr = pof_sat.read(1)

inv_arr = inventory.read(1)

Wall time: 4min 41s


In [3]:
%%time
#Create Data Frame <- 2D Arrays have to transformed to 1D
df = pd.DataFrame()

df['lat'] = lat_arr.ravel()
df['long'] = long_arr.ravel()

df['elevation'] = dem_arr.ravel()
df['slope'] = slp_arr.ravel()
df['aspect'] = asp_arr.ravel()
df['curv_plan'] = curv_plan_arr.ravel()
df['curv_prof'] = curv_prof_arr.ravel()
df['facc'] = facc_arr.ravel()
df['twi'] = twi_arr.ravel()

df['soil'] = soil_arr.ravel()
df['lulc'] = lulc_arr.ravel()

#df['rain_ant'] = rain_ant_arr.ravel()
#df['rain_event'] = rain_event_arr.ravel()

#df['pof_event'] = pof_event_arr.ravel()
#df['pof_dry'] = pof_dry_arr.ravel()
#df['pof_sat'] = pof_sat_arr.ravel()

df['response'] = inv_arr.ravel()

Wall time: 2.14 s


In [4]:
for i in df.columns:
    print(str(i) + ': ' + str(df[(df[i] == -9999)].shape[0]))

lat: 0
long: 0
elevation: 10104420
slope: 10104420
aspect: 10104478
curv_plan: 10104420
curv_prof: 10104420
facc: 10104420
twi: 10104478
soil: 10223986
lulc: 10135371
response: 0


In [5]:
#Replace NoData with Numpy np.nan
df_nan = df.replace(-9999, np.nan)
print("Cells in Raster Extent: ", df.shape[0])
df_nan = df_nan.dropna()
df_nan = df_nan.reset_index(drop=True)
print("Cells with Values: ", df_nan.shape[0])

#Transform FAcc to LOG10
df_nan['facc'] = np.log10(df_nan['facc'])
from numpy import inf
df_nan = df_nan.replace(-inf, 0)

df_nan.head()

Cells in Raster Extent:  30294888
Cells with Values:  20036260


C:\anaconda3\envs\geo_py38\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,lat,long,elevation,slope,aspect,curv_plan,curv_prof,facc,twi,soil,lulc,response
0,4685862.5,408812.5,2520.639893,1.511818,142.706085,0.011680,-0.000727,1.69897,7.546706,12.0,7.0,0.0
1,4685862.5,408817.5,2520.570068,1.111485,145.510040,-0.003146,-0.001120,1.39794,7.161282,12.0,7.0,0.0
2,4685862.5,408822.5,2520.530029,1.054280,135.000000,0.114092,0.000303,1.39794,7.214133,12.0,7.0,0.0
3,4685862.5,408827.5,2520.439941,2.101016,101.010818,0.020339,0.007637,0.00000,-8.207567,12.0,4.0,0.0
4,4685862.5,408832.5,2520.169922,7.283532,104.028282,0.086840,0.028588,0.00000,-9.455721,12.0,4.0,0.0


In [6]:
df_nan[(df_nan["response"] == 1)].shape[0]

976

In [7]:
#Separating into True and False groups of the Response attribute
true_cells = df_nan[df_nan.response == True]
false_cells = df_nan[df_nan.response == False]
print("True cells: ", true_cells.shape[0], "\nFalse cells: ", false_cells.shape[0])

#Sampling from False
false_sample = false_cells.sample(n = true_cells.shape[0], random_state = 0)
#false_sample = false_cells.sample(n = 5000, random_state = 0)
print(false_sample.shape[0], " have been sampled from the false cells.")

#Creating the DataFrame for the data-driven model
df_dd = pd.concat([true_cells, false_sample])
print("Therefore, the final DataFrame for the data-driven model has:", df_dd.shape[0], "observations.")
df_dd.head()

True cells:  976 
False cells:  20035284
976  have been sampled from the false cells.
Therefore, the final DataFrame for the data-driven model has: 1952 observations.


,lat,long,elevation,slope,aspect,curv_plan,curv_prof,facc,twi,soil,lulc,response
1620095,4682432.5,415802.5,1662.609985,28.607676,209.432648,-0.030294,0.019568,4.197969,10.272434,12.0,5.0,1.0
2225065,4681897.5,403107.5,1343.520020,36.750950,197.294296,-0.000409,0.000365,3.113943,7.462131,7.0,6.0,1.0
2318639,4681822.5,419452.5,1441.219971,23.992632,183.993134,0.048947,0.007482,3.021189,7.766060,12.0,5.0,1.0
2354712,4681792.5,419462.5,1428.079956,23.833015,189.248627,0.020361,0.001992,2.977724,7.673494,12.0,5.0,1.0
2365749,4681782.5,414537.5,1455.969971,52.255924,176.982315,0.003440,0.027422,1.698970,3.655993,9.0,8.0,1.0


In [8]:
#Export the data set
df_dd.to_csv (r'C:\Users\malva\Thesis\Files\Samples\Bergueda_Sample.csv', index=False, header=True)
#df_dd.to_csv (r'C:\Users\malva\Thesis\Files\Samples\ValdAran_Sample_5391.csv', index=False, header=True)